In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np

In [2]:
CUE_ON_T = 1.0
SAMPLE_ON_T = SACCADE_ON_TIME = 3.0
MNM_END_TIME = 3.0
SUBREGIONS = {'Mid-Dorsal': 'MD', 'Posterior-Ventral': 'PV',
              'Anterior-Dorsal': 'AD', 'Posterior-Dorsal': 'PD', 'Anterior-Ventral': 'AV'}

def mat_to_df(dg, monkey, area, num_stimuli=1, numeric_cell=False, ):
    rows = []
    cells = []
    for c in range(dg.shape[0]):
        c_num = str(c + 1).zfill(3)
        cell_name = f'{monkey}-{area}-{c_num}'
        # cells.append([c + 1 if numeric_cell else cell_name, monkey, area])
        for p in range(dg.shape[1]):
            feature_names = dg[c][p][0].dtype.names
            for t in range(dg[c, p].shape[1]):
                cue_on_t = dg[c, p][0, t]['Cue_onT'][0][0]
                ts = dg[c, p][0, t]['TS'].flatten()
                if num_stimuli == 1:
                    ts = ts - cue_on_t + CUE_ON_T
                    ts = np.array([t for t in ts if t >= 0 and t <=
                                  SACCADE_ON_TIME + 1.0], dtype=np.float32)
                    if (len(ts) > 0):
                        rows.append([cell_name, p, t, ts])  # , cue_on_t])
                else:
                    sample_on_t = dg[c, p][0, t]['Sample_onT'][0][0]
                    ts2 = np.array([SAMPLE_ON_T + t - sample_on_t for t in ts if t >
                                   sample_on_t and t <= sample_on_t + MNM_END_TIME], dtype=np.float32)
                    ts = np.array([t - cue_on_t + CUE_ON_T for t in ts if t >= cue_on_t -
                                  CUE_ON_T and t <= cue_on_t + SAMPLE_ON_T - CUE_ON_T], dtype=np.float32)
                    ts = np.concatenate((ts, ts2))
                    try:
                        ismatch = dg[c, p][0, t]['IsMatch'][0][0]
                        if len(ts) > 0:
                            rows.append([c + 1, p, t, ts, ismatch])
                    except:
                        pass
 
    df = pd.DataFrame(rows, columns=[
                      'cell', 'position', 'trial', 'ts'] + ([] if num_stimuli == 1 else ['ismatch']))
    # df = df.set_index(['cell', 'position', 'trial'])
    # , pd.DataFrame(cells, columns=['cell', 'monkey', 'area']).set_index('cell')
    return df

In [3]:
def get_cells(df, style='monkey-area'):
    cells = df.cell.unique()
    cells = pd.DataFrame(cells, columns=['cell'])
    cells['monkey'] = cells.cell.apply(lambda x: x.split('-')[0])
    cells['area'] = cells.cell.apply(lambda x: x.split('-')[1])
    if 'subregion' in style:
        cells['subregion'] = cells.cell.apply(lambda x: x.split('-')[2])
    if 'phase' in style:
        cells['phase'] = cells.cell.apply(lambda x: x.split('-')[3])
    cells = cells.set_index('cell')
    return cells


In [6]:
def load_mnm_feature():
    asd_mat = loadmat(
        '../../../data/MNM_original_data/all_feature_data.mat')['all_feature_data']
    asi_mat = loadmat(
        '../../../data/MNM_original_data/all_feature_info.mat')['all_feature_info']

    asd_mat = asd_mat[:, :8]
    asi_mat = asi_mat[:, [0, 3, 4]]
    extract = np.vectorize(lambda x: x[0])
    asi_mat = extract(asi_mat)

    asi_df = pd.DataFrame(asi_mat, columns=['monkey', 'phase', 'subregion'])
    asi_df.monkey = asi_df.monkey.apply(lambda x: x[0:3].upper())
    asi_df.subregion = asi_df.subregion.apply(lambda x: SUBREGIONS[x].upper())

    asd_df = mat_to_df(asd_mat, '<REPLACE>', 'PFC',
                        num_stimuli=2, numeric_cell=True)

    df = pd.merge(asd_df, asi_df, left_on='cell', right_index=True)
    df['cell'] = df.apply(lambda r: '-'.join([r.monkey, 'PFC',
                          str(r.subregion), r.phase, str(r.cell)]), axis=1)
    cells = get_cells(df, style='monkey-area-subregion-phase')
    df = df.drop(columns=['monkey', 'subregion', 'phase'])

    return {'raw_df': df.set_index(['cell', 'position', 'trial']), 'cells': cells}

In [7]:
df_dict = load_mnm_feature()

In [8]:
raw_df = pd.DataFrame(df_dict['raw_df'])

In [9]:
cells = pd.DataFrame(df_dict['cells'])

In [10]:
# Resetting the index to move 'cell', 'position', and 'trial' back to columns
raw_df = raw_df.reset_index()

# Verify the change by printing the columns
# raw_df.sample(10)

In [11]:
df = pd.merge(raw_df, cells, on='cell', how='inner')
df.sample(3)

,cell,position,trial,ts,ismatch,monkey,area,subregion,phase
246366,517-PFC-PD-POST-2211,5,11,"[0.12185, 1.626125, 2.1231, 2.373875, 2.557375...",0.0,517,PFC,PD,POST
207260,508-PFC-MD-POST-1797,3,0,"[1.149525, 2.444025, 3.2022, 4.755125]",0.0,508,PFC,MD,POST
254326,619-PFC-PV-POST-2306,4,9,"[0.04865, 0.52065, 0.53145, 2.328925, 2.43565,...",0.0,619,PFC,PV,POST


In [12]:
lengths = df.ts.apply(lambda x: len(x))
# lengths.value_counts()

In [13]:
filtered_df = df
filtered_df['ts_length'] = filtered_df['ts'].apply(len)
average_ts_length = filtered_df.groupby('cell')['ts_length'].mean().reset_index()
sum_ts_length = filtered_df.groupby('cell')['ts_length'].sum().reset_index()
use_cell = average_ts_length[(average_ts_length.ts_length >= 100) & (sum_ts_length.ts_length >= 500)].cell

In [14]:
filtered_df = filtered_df[filtered_df['cell'].isin(use_cell)]
filtered_df.shape

(10806, 10)

In [15]:
pre_df = filtered_df[filtered_df.phase == "PRE"]
post_df = filtered_df[filtered_df.phase == "POST"]
pre_df.shape, post_df.shape

((6411, 10), (4395, 10))

In [16]:
import torch

def get_soft_one_hot(pos):
    return torch.tensor(
        [
            0.6 if pos == x else (0.2 if abs(x - pos) ==
                                  1 or abs(x - pos) == 7 else 0)
            for x in range(1, 9)
        ]
    )


def create_y(l, func):
    return torch.hstack(
        [
            torch.stack([func(y) for y in l]),
            torch.tensor(l).unsqueeze(1),
        ]
    )

def window_data(df, NUM_OF_SAMPLES, WINDOW_STRIDE, WINDOW_WIDTH, NUM_POSITIONS, NUM_OF_CELLS):
    df = df.sort_values(['cell', 'position']).reset_index(drop=True)

    window_data = []
    for start in np.arange(1, 3.5, WINDOW_STRIDE):
        window_data.append(df.ts.apply(lambda x: (
            (x >= start) & (x < start + WINDOW_WIDTH)).sum()).array)

    window_d_t = torch.tensor(window_data).T
    BINS = window_d_t.shape[1]

    X = torch.zeros((NUM_OF_SAMPLES * NUM_POSITIONS,
                    NUM_OF_CELLS * BINS), dtype=torch.float32)
    y = []
    for p in range(1, NUM_POSITIONS+1):
        y += [p] * NUM_OF_SAMPLES
        cells_idxs = df[df.position == p-1].groupby("cell", sort=False).apply(
            lambda x: x.index)
        for i, cell_idxs in enumerate(cells_idxs):
            idxs = np.random.choice(cell_idxs, NUM_OF_SAMPLES)
            for j, idx in enumerate(idxs):
                X[(p - 1) * NUM_OF_SAMPLES + j, i *
                  BINS: (i + 1) * BINS] = window_d_t[idx]

    return X, create_y(y, get_soft_one_hot)

In [17]:
from sklearn.model_selection import train_test_split 

NUM_OF_CELLS = pre_df.cell.nunique()
NUM_OF_SAMPLES = 40
WINDOW_STRIDE = 0.4
WINDOW_WIDTH = 0.4
NUM_POSITIONS = 2

X, y = window_data(pre_df, NUM_OF_SAMPLES, WINDOW_STRIDE, WINDOW_WIDTH, NUM_POSITIONS, NUM_OF_CELLS)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) 

In [18]:
X.shape

torch.Size([80, 378])

In [19]:
y.shape

torch.Size([80, 9])

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleFNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_size = X.shape[1]
num_classes = y_train_labels.max().item() + 1

model = SimpleFNN(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/20], Loss: 1.8824, Accuracy: 40.62%
Epoch [2/20], Loss: 1.3893, Accuracy: 60.94%
Epoch [3/20], Loss: 0.6003, Accuracy: 71.88%
Epoch [4/20], Loss: 0.3930, Accuracy: 81.25%
Epoch [5/20], Loss: 0.4483, Accuracy: 78.12%
Epoch [6/20], Loss: 0.1513, Accuracy: 93.75%
Epoch [7/20], Loss: 0.2052, Accuracy: 90.62%
Epoch [8/20], Loss: 0.0948, Accuracy: 95.31%
Epoch [9/20], Loss: 0.0598, Accuracy: 98.44%
Epoch [10/20], Loss: 0.0853, Accuracy: 96.88%
Epoch [11/20], Loss: 0.0345, Accuracy: 98.44%
Epoch [12/20], Loss: 0.0279, Accuracy: 100.00%
Epoch [13/20], Loss: 0.0333, Accuracy: 100.00%
Epoch [14/20], Loss: 0.0220, Accuracy: 100.00%
Epoch [15/20], Loss: 0.0131, Accuracy: 100.00%
Epoch [16/20], Loss: 0.0112, Accuracy: 100.00%
Epoch [17/20], Loss: 0.0139, Accuracy: 100.00%
Epoch [18/20], Loss: 0.0128, Accuracy: 100.00%
Epoch [19/20], Loss: 0.0094, Accuracy: 100.00%
Epoch [20/20], Loss: 0.0068, Accuracy: 100.00%
Training Finished.


In [21]:
# Evaluation function
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0064, Test Accuracy: 100.00%
Final Test Loss: 0.0064, Final Test Accuracy: 100.00%


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleCNN(nn.Module):
    def __init__(self, num_cells, bins, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_cells, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

BINS = 7
num_cells = X.shape[1] // BINS
bins = BINS
num_classes = y_train_labels.max().item() + 1

model = SimpleCNN(num_cells, bins, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), num_cells, bins)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')

Epoch [1/30], Loss: 1.1972, Accuracy: 42.19%
Epoch [2/30], Loss: 0.8183, Accuracy: 57.81%
Epoch [3/30], Loss: 0.7072, Accuracy: 59.38%
Epoch [4/30], Loss: 0.6063, Accuracy: 73.44%
Epoch [5/30], Loss: 0.5579, Accuracy: 76.56%
Epoch [6/30], Loss: 0.4928, Accuracy: 82.81%
Epoch [7/30], Loss: 0.4674, Accuracy: 78.12%
Epoch [8/30], Loss: 0.3945, Accuracy: 84.38%
Epoch [9/30], Loss: 0.3839, Accuracy: 84.38%
Epoch [10/30], Loss: 0.3229, Accuracy: 85.94%
Epoch [11/30], Loss: 0.3078, Accuracy: 84.38%
Epoch [12/30], Loss: 0.2408, Accuracy: 93.75%
Epoch [13/30], Loss: 0.2329, Accuracy: 95.31%
Epoch [14/30], Loss: 0.1947, Accuracy: 87.50%
Epoch [15/30], Loss: 0.1750, Accuracy: 95.31%
Epoch [16/30], Loss: 0.1334, Accuracy: 96.88%
Epoch [17/30], Loss: 0.1130, Accuracy: 96.88%
Epoch [18/30], Loss: 0.0930, Accuracy: 100.00%
Epoch [19/30], Loss: 0.0788, Accuracy: 100.00%
Epoch [20/30], Loss: 0.0645, Accuracy: 100.00%
Epoch [21/30], Loss: 0.0554, Accuracy: 100.00%
Epoch [22/30], Loss: 0.0442, Accuracy: 

In [23]:
# Evaluation function
def evaluate_model(model, test_loader, criterion, num_cells, bins):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), num_cells, bins)  # Reshape the inputs to match the CNN input size
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, num_cells, bins)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0083, Test Accuracy: 100.00%
Final Test Loss: 0.0083, Final Test Accuracy: 100.00%


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, num_features, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.relu(x[:, -1, :])
        x = self.fc(x)
        return x

num_features = BINS
sequence_length = len(pre_df.cell.unique())
hidden_size = 64
num_layers = 2
num_classes = y_train_labels.max().item() + 1

model = LSTMModel(num_features, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), sequence_length, num_features)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/100], Loss: 1.1174, Accuracy: 39.06%
Epoch [2/100], Loss: 1.0535, Accuracy: 43.75%
Epoch [3/100], Loss: 0.9959, Accuracy: 45.31%
Epoch [4/100], Loss: 0.9409, Accuracy: 59.38%
Epoch [5/100], Loss: 0.8858, Accuracy: 56.25%
Epoch [6/100], Loss: 0.8416, Accuracy: 56.25%
Epoch [7/100], Loss: 0.8066, Accuracy: 56.25%
Epoch [8/100], Loss: 0.7778, Accuracy: 56.25%
Epoch [9/100], Loss: 0.7574, Accuracy: 56.25%
Epoch [10/100], Loss: 0.7411, Accuracy: 56.25%
Epoch [11/100], Loss: 0.7282, Accuracy: 56.25%
Epoch [12/100], Loss: 0.7179, Accuracy: 56.25%
Epoch [13/100], Loss: 0.7084, Accuracy: 56.25%
Epoch [14/100], Loss: 0.7003, Accuracy: 56.25%
Epoch [15/100], Loss: 0.6919, Accuracy: 56.25%
Epoch [16/100], Loss: 0.6828, Accuracy: 56.25%
Epoch [17/100], Loss: 0.6755, Accuracy: 56.25%
Epoch [18/100], Loss: 0.6667, Accuracy: 57.81%
Epoch [19/100], Loss: 0.6545, Accuracy: 65.62%
Epoch [20/100], Loss: 0.6448, Accuracy: 68.75%
Epoch [21/100], Loss: 0.6336, Accuracy: 65.62%
Epoch [22/100], Loss: 

Epoch [23/100], Loss: 0.6122, Accuracy: 78.12%
Epoch [24/100], Loss: 0.5945, Accuracy: 75.00%
Epoch [25/100], Loss: 0.5792, Accuracy: 76.56%
Epoch [26/100], Loss: 0.5760, Accuracy: 85.94%
Epoch [27/100], Loss: 0.6212, Accuracy: 68.75%
Epoch [28/100], Loss: 0.5876, Accuracy: 68.75%
Epoch [29/100], Loss: 0.5484, Accuracy: 85.94%
Epoch [30/100], Loss: 0.5367, Accuracy: 85.94%
Epoch [31/100], Loss: 0.5112, Accuracy: 82.81%
Epoch [32/100], Loss: 0.5175, Accuracy: 76.56%
Epoch [33/100], Loss: 0.4927, Accuracy: 84.38%
Epoch [34/100], Loss: 0.4842, Accuracy: 87.50%
Epoch [35/100], Loss: 0.4611, Accuracy: 89.06%
Epoch [36/100], Loss: 0.4523, Accuracy: 87.50%
Epoch [37/100], Loss: 0.4320, Accuracy: 89.06%
Epoch [38/100], Loss: 0.4179, Accuracy: 90.62%
Epoch [39/100], Loss: 0.4412, Accuracy: 78.12%
Epoch [40/100], Loss: 0.3891, Accuracy: 92.19%
Epoch [41/100], Loss: 0.3853, Accuracy: 90.62%
Epoch [42/100], Loss: 0.3602, Accuracy: 90.62%
Epoch [43/100], Loss: 0.3420, Accuracy: 90.62%
Epoch [44/100

In [25]:
# Evaluation function for LSTM model
def evaluate_model(model, test_loader, criterion, sequence_length, num_features):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), sequence_length, num_features)  # Reshape inputs
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, sequence_length, num_features)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0211, Test Accuracy: 100.00%
Final Test Loss: 0.0211, Final Test Accuracy: 100.00%


In [26]:
from sklearn.model_selection import train_test_split 

NUM_OF_CELLS = post_df.cell.nunique()
NUM_OF_SAMPLES = 40
WINDOW_STRIDE = 0.4
WINDOW_WIDTH = 0.4
NUM_POSITIONS = 2

X, y = window_data(post_df, NUM_OF_SAMPLES, WINDOW_STRIDE, WINDOW_WIDTH, NUM_POSITIONS, NUM_OF_CELLS)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) 

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleFNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_size = X.shape[1]
num_classes = y_train_labels.max().item() + 1

model = SimpleFNN(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/20], Loss: 3.9676, Accuracy: 40.62%
Epoch [2/20], Loss: 1.2824, Accuracy: 54.69%
Epoch [3/20], Loss: 1.0240, Accuracy: 65.62%
Epoch [4/20], Loss: 0.8179, Accuracy: 59.38%
Epoch [5/20], Loss: 0.3982, Accuracy: 78.12%
Epoch [6/20], Loss: 0.6026, Accuracy: 67.19%
Epoch [7/20], Loss: 0.2723, Accuracy: 92.19%
Epoch [8/20], Loss: 0.4298, Accuracy: 76.56%
Epoch [9/20], Loss: 0.2262, Accuracy: 92.19%
Epoch [10/20], Loss: 0.2751, Accuracy: 89.06%
Epoch [11/20], Loss: 0.1675, Accuracy: 95.31%
Epoch [12/20], Loss: 0.1615, Accuracy: 95.31%
Epoch [13/20], Loss: 0.1333, Accuracy: 98.44%
Epoch [14/20], Loss: 0.1010, Accuracy: 95.31%
Epoch [15/20], Loss: 0.0984, Accuracy: 96.88%
Epoch [16/20], Loss: 0.0660, Accuracy: 100.00%
Epoch [17/20], Loss: 0.0684, Accuracy: 100.00%
Epoch [18/20], Loss: 0.0567, Accuracy: 100.00%
Epoch [19/20], Loss: 0.0443, Accuracy: 100.00%
Epoch [20/20], Loss: 0.0449, Accuracy: 100.00%
Training Finished.


In [28]:
# Evaluation function
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0403, Test Accuracy: 100.00%
Final Test Loss: 0.0403, Final Test Accuracy: 100.00%


In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleCNN(nn.Module):
    def __init__(self, num_cells, bins, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_cells, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

BINS = 7
num_cells = X.shape[1] // BINS
bins = BINS
num_classes = y_train_labels.max().item() + 1

model = SimpleCNN(num_cells, bins, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), num_cells, bins)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')

Epoch [1/30], Loss: 0.9112, Accuracy: 54.69%
Epoch [2/30], Loss: 0.7836, Accuracy: 56.25%
Epoch [3/30], Loss: 0.7085, Accuracy: 56.25%
Epoch [4/30], Loss: 0.6664, Accuracy: 59.38%
Epoch [5/30], Loss: 0.6421, Accuracy: 65.62%
Epoch [6/30], Loss: 0.6124, Accuracy: 71.88%
Epoch [7/30], Loss: 0.5870, Accuracy: 65.62%
Epoch [8/30], Loss: 0.5657, Accuracy: 65.62%
Epoch [9/30], Loss: 0.5235, Accuracy: 82.81%
Epoch [10/30], Loss: 0.5117, Accuracy: 84.38%
Epoch [11/30], Loss: 0.4693, Accuracy: 87.50%
Epoch [12/30], Loss: 0.4443, Accuracy: 82.81%
Epoch [13/30], Loss: 0.4062, Accuracy: 93.75%
Epoch [14/30], Loss: 0.3566, Accuracy: 92.19%
Epoch [15/30], Loss: 0.3439, Accuracy: 89.06%
Epoch [16/30], Loss: 0.2781, Accuracy: 96.88%
Epoch [17/30], Loss: 0.2669, Accuracy: 96.88%
Epoch [18/30], Loss: 0.2397, Accuracy: 95.31%
Epoch [19/30], Loss: 0.1950, Accuracy: 98.44%
Epoch [20/30], Loss: 0.1700, Accuracy: 96.88%
Epoch [21/30], Loss: 0.1325, Accuracy: 100.00%
Epoch [22/30], Loss: 0.1179, Accuracy: 100

In [30]:
# Evaluation function
def evaluate_model(model, test_loader, criterion, num_cells, bins):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), num_cells, bins)  # Reshape the inputs to match the CNN input size
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, num_cells, bins)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0147, Test Accuracy: 100.00%
Final Test Loss: 0.0147, Final Test Accuracy: 100.00%


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, num_features, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.relu(x[:, -1, :])
        x = self.fc(x)
        return x

num_features = BINS
sequence_length = len(post_df.cell.unique())
hidden_size = 64
num_layers = 2
num_classes = y_train_labels.max().item() + 1

model = LSTMModel(num_features, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), sequence_length, num_features)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/100], Loss: 1.1337, Accuracy: 26.56%
Epoch [2/100], Loss: 1.0835, Accuracy: 54.69%
Epoch [3/100], Loss: 1.0371, Accuracy: 54.69%
Epoch [4/100], Loss: 0.9900, Accuracy: 54.69%
Epoch [5/100], Loss: 0.9486, Accuracy: 54.69%
Epoch [6/100], Loss: 0.9094, Accuracy: 54.69%
Epoch [7/100], Loss: 0.8784, Accuracy: 54.69%
Epoch [8/100], Loss: 0.8535, Accuracy: 54.69%
Epoch [9/100], Loss: 0.8326, Accuracy: 54.69%
Epoch [10/100], Loss: 0.8141, Accuracy: 54.69%
Epoch [11/100], Loss: 0.7983, Accuracy: 54.69%
Epoch [12/100], Loss: 0.7846, Accuracy: 54.69%
Epoch [13/100], Loss: 0.7729, Accuracy: 54.69%
Epoch [14/100], Loss: 0.7632, Accuracy: 54.69%
Epoch [15/100], Loss: 0.7548, Accuracy: 54.69%
Epoch [16/100], Loss: 0.7487, Accuracy: 54.69%
Epoch [17/100], Loss: 0.7427, Accuracy: 54.69%
Epoch [18/100], Loss: 0.7379, Accuracy: 54.69%
Epoch [19/100], Loss: 0.7338, Accuracy: 54.69%
Epoch [20/100], Loss: 0.7298, Accuracy: 54.69%
Epoch [21/100], Loss: 0.7257, Accuracy: 54.69%
Epoch [22/100], Loss: 

Epoch [26/100], Loss: 0.7068, Accuracy: 54.69%
Epoch [27/100], Loss: 0.7112, Accuracy: 54.69%
Epoch [28/100], Loss: 0.7051, Accuracy: 54.69%
Epoch [29/100], Loss: 0.7023, Accuracy: 54.69%
Epoch [30/100], Loss: 0.6973, Accuracy: 54.69%
Epoch [31/100], Loss: 0.6869, Accuracy: 56.25%
Epoch [32/100], Loss: 0.6835, Accuracy: 64.06%
Epoch [33/100], Loss: 0.6795, Accuracy: 56.25%
Epoch [34/100], Loss: 0.6674, Accuracy: 70.31%
Epoch [35/100], Loss: 0.6566, Accuracy: 67.19%
Epoch [36/100], Loss: 0.6459, Accuracy: 68.75%
Epoch [37/100], Loss: 0.6864, Accuracy: 60.94%
Epoch [38/100], Loss: 0.6766, Accuracy: 65.62%
Epoch [39/100], Loss: 0.6984, Accuracy: 54.69%
Epoch [40/100], Loss: 0.6899, Accuracy: 54.69%
Epoch [41/100], Loss: 0.6780, Accuracy: 54.69%
Epoch [42/100], Loss: 0.6781, Accuracy: 78.12%
Epoch [43/100], Loss: 0.6684, Accuracy: 73.44%
Epoch [44/100], Loss: 0.6645, Accuracy: 57.81%
Epoch [45/100], Loss: 0.6510, Accuracy: 62.50%
Epoch [46/100], Loss: 0.6384, Accuracy: 71.88%
Epoch [47/100

In [32]:
# Evaluation function for LSTM model
def evaluate_model(model, test_loader, criterion, sequence_length, num_features):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), sequence_length, num_features)  # Reshape inputs
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, sequence_length, num_features)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0732, Test Accuracy: 100.00%
Final Test Loss: 0.0732, Final Test Accuracy: 100.00%
